# 第6章 使用一等函数实现设计模式

## 6.1 案例分析：重构“策略”模式
如果合理利用作为一等对象的函数，某些设计模式可以简化。

### 6.1.1 经典的“策略”模式
如果不清楚这个模式的话可以自己找一下资料了解。

下面是一个电商促销打折的策略模式

In [6]:
from abc import ABC, abstractmethod
from collections import namedtuple

In [2]:
class LineItem(object):  # 一般化的商品类
    
    def __init__(self, product:str, quantity:int, price:float):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self) -> float:
        return self.quantity * self.price

In [15]:
class Order(object):  # 订单上下文
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        self.__total = None
    
    @property
    def total(self):
        """计算订单总价格
        """
        total = sum(item.total() for item in self.cart)
        self.__total = total
        return total

    def due(self):
        if self.promotion is None:
            dicount = 0
        else:
            discount = self.promotion.discount(self)  # 传入上下文
        return self.total - discount
    
    def __repr__(self):
        fmt = "<Order total {:.2f} due {:.2f}>"
        return fmt.format(self.total, self.due())

In [7]:
class Promotion(ABC):  # 抽象基类
    
    @abstractmethod
    def discount(self, order):
        pass

In [8]:
class FidelityPromo(Promotion):
    
    def discount(self, order):
        """ 积分超过1000, 提供5%的折扣
        """
        return order.total * 0.05 if order.customer.fidelity >= 1000 else 0

In [20]:
class BulkItemPromo(Promotion):
    """单个商品为20个或以上时提供10%折扣
    """
    
    def discount(self, order):
        discount = sum(map(lambda item: 0.1 * item.total() if item.quantity >= 20 else 0, order.cart))
        return discount

In [10]:
class LargeOrderPromo(Promotion):
    """订单中不同商品达到10或以上提供7%折扣
    """
    
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        return order.total * 0.07 if len(distinct_items) >= 10 else 0 

In [11]:
# 创建Customer类
Customer = namedtuple("Customer", "name fidelity")

In [16]:
joe = Customer("John", 0)
ann = Customer("ann", 1100)
cart = [LineItem("banana", 4, .5),
       LineItem("apple", 10, 1.5),
       LineItem("watermelon", 5, 5.0)]
# 创建订单
o1 = Order(joe, cart, FidelityPromo())
o1

<Order total 42.00 due 42.00>

In [18]:
o2 = Order(ann, cart, FidelityPromo())
o2

<Order total 42.00 due 39.90>

In [21]:
banana_cart = [LineItem("banana", 30, 0.5),
              LineItem("apple", 10, 1.5)]
Order(joe, banana_cart, BulkItemPromo())

<Order total 30.00 due 28.50>

In [22]:
long_order = [LineItem(str(item_code), 1, 1.0) for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total 10.00 due 9.30>

从上面这个例子来说, 每一种折扣其实只是实现了一个方法, 所以对于将函数视作一等对象的语言来说, 可以完全不去写那个抽象基类

In [27]:
# 看这个例子
def fidelity_promo(order):
    return order.total * 0.05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
    return sum(map(lambda item: 0.1 * item.total() if item.quantity >= 20 else 0, order.cart))

def large_order_promo(order):
    distinct_items = {item.product for item in order.cart}
    return order.total * 0.07 if len(distinct_items) >= 10 else 0 

class Order(object):  # 订单上下文
    
    def __init__(self, customer, cart, promotion = None):
        self.customer = customer
        self.cart = cart
        self.promotion = promotion
        self.__total = None
    
    @property
    def total(self):
        """计算订单总价格
        """
        total = sum(item.total() for item in self.cart)
        self.__total = total
        return total

    def due(self):
        if self.promotion is None:
            dicount = 0
        else:
            discount = self.promotion(self)  # 只需改动这一行
        return self.total - discount
    
    def __repr__(self):
        fmt = "<Order total {:.2f} due {:.2f}>"
        return fmt.format(self.total, self.due())

上面这样重构, 除了代码简洁了, 还有就是减少了不必要的对象创建, 让函数成为一个共享元

In [28]:
Order(ann, cart, fidelity_promo)

<Order total 42.00 due 39.90>

### 6.1.3 选择最佳策略：简单的方式

In [29]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]
def best_promo(order):
    """选择最大折扣的策略
    """
    return max(promo(order) for promo in promos)

In [30]:
# 测试
Order(joe, long_order, best_promo)

<Order total 10.00 due 9.30>

In [31]:
Order(joe, banana_cart, best_promo)

<Order total 30.00 due 28.50>

In [32]:
Order(ann, cart, best_promo)

<Order total 42.00 due 39.90>

### 6.1.4 找出模块中的全部策略
6.1.3中的例子promos是一个列表, 每次新增加策略的时候如果忘了在promos中添加这个策略, 那么best_promo就不会考虑这个策略。所以, 为了解决这个问

题, 我们 需要用到内置函数globals

In [34]:
promos = [globals()[name] for name in globals() if name.endswith("_promo") and name != "best_promo"]
def best_promo(order):
    """选择最大折扣的策略
    """
    return max(promo(order) for promo in promos)

单独从另一个模块(promotions)导入, 需要用到inspect模块
```python
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]
```

## 6.2 命令模式
一般命令模式的类一般只有一个方法, 和上面的策略模式差不多, 都可以用函数重构